In [2]:
import numpy as np
from scipy import stats
import pandas as pd
import sys
import os
import re

# 1. Remove Outliers

In [11]:
route = '672'
direction = 0

df = pd.read_csv('output/{}_{}X.csv'.format(route, direction))
df_num = df.iloc[:,5:]
count = 1

for idx, row in df_num.iterrows():
    for i in range(len(row)):
        z = ( row[i] - row.mean() )/row.std()
        if abs(z) > 3:
            df_num.iloc[idx,i] = None

df_input = pd.concat([df.iloc[:,:5], df_num], axis=1, sort=False)
df_input.head()

,StopName,StopUID,Direction,RouteName,StopSequence,2020-6-22_0700,2020-6-22_0705,2020-6-22_0710,2020-6-22_0715,2020-6-22_0720,...,2020-6-22_0815,2020-6-22_0820,2020-6-22_0825,2020-6-22_0830,2020-6-22_0835,2020-6-22_0840,2020-6-22_0845,2020-6-22_0850,2020-6-22_0855,mean
0,民生社區活動中心,TPE38902,0,672,39,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,聯合二村,TPE38901,0,672,38,24.0,37.0,37.0,37.0,37.0,...,35.0,47.0,47.0,47.0,48.0,48.0,48.0,NaN,53.0,41.434783
2,介壽國中,TPE38900,0,672,37,41.0,54.0,54.0,54.0,54.0,...,61.0,75.0,75.0,75.0,83.0,83.0,83.0,61.0,61.0,62.590909
3,公教住宅,TPE38899,0,672,36,NaN,49.0,49.0,49.0,49.0,...,NaN,52.0,52.0,52.0,57.0,57.0,57.0,65.0,65.0,53.090909
4,長庚醫院,TPE38898,0,672,35,64.0,134.0,134.0,134.0,134.0,...,211.0,201.0,201.0,NaN,201.0,201.0,NaN,164.0,164.0,160.300000


# 2. Calculate Avg. by Hour

In [15]:
class timeSpendingHrAvg():
    def __init__(self, stop_file, df_input, output_folder, route, direction, timespending_file=None):
        self.stop_file = stop_file
        self.timespending_file = timespending_file
        self.output_folder = output_folder
        self.route = route
        self.direction = direction
        self.df_input = df_input
        self.output = None
    
    
    def route_extractor(self, routeName, direction):
        route = pd.read_csv(self.stop_file)
        single_route = route[(route['RouteName'] == routeName) & (route['Direction'] == direction)].drop_duplicates()
        if single_route.shape[0] == 0:
            single_route = route[(route['RouteName'] == int(routeName)) & (route['Direction'] == direction)].drop_duplicates()
        
        return single_route 
   

    def route_frame(self, directory, filename):
        single_route = self.route_extractor(self.route, self.direction)
        result = single_route[['StopName','StopUID','Direction','RouteName','StopSequence']] \
        .sort_values(by='StopSequence', ascending=False)
        return result

    
    def hourly_est_avg(self): #with csv
        
        
        if self.timespending_file == None:
            df_matric = self.df_input
        else:
            df_matric = pd.read_csv(self.timespending_file)    
        
        est_hour = {}
        hour_list = ['00','01','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
        #hour_list = []
        for hour in hour_list:
            hour_selecter = re.compile('{}[0-9][0-9]'.format(hour))
            stop_est = {}
            for stopseq in range(len(df_matric)):
                time_est = []
                name = df_matric.loc[stopseq,'StopName']
                for column in df_matric.columns: #get the columns of assigned hour
                    if re.match(hour_selecter, column[-4:]):
                        time_est.append(df_matric.loc[stopseq,column])

                zeros = time_est.count(0)
                try:
                    zero_ratio = zeros/len(time_est)
                except ZeroDivisionError:
                    zero_ratio = 0
                    pass
                    #print(hour,name)

                remove_zeros = list(filter(lambda a: a != 0, time_est))
                remove_zeros = list(filter(lambda a: a != 1, remove_zeros))
                n_est = np.array(remove_zeros)
                n_est = n_est[~np.isnan(n_est)]

                if zero_ratio < 0.4:
                    result = round(np.mean(n_est),2)
                elif zero_ratio >= 0.4:
                    try:
                        result = stats.mode(n_est)[0][0]
                    except IndexError:
                        #print(hour,stopseq)
                        result = None
                else:
                    result = None

                stop_est[name] = result
            est_hour[hour] = stop_est #dict format

        #return DataFrame format
        return pd.DataFrame.from_dict(est_hour).reset_index()
    
    def format_df(self):
        df_hr_avg = self.hourly_est_avg()
        #display(df_hr_avg)
        #display(self.route_extractor(self.route, self.direction))
        self.output = self.route_frame(
                     directory='bus-data', 
                     filename='{}-eta_2020-6-22_0730.csv'.format(self.route)) \
                     .merge(df_hr_avg,left_on='StopName',right_on='index').drop('index',axis=1)
        self.output.to_csv('{}/{}_{}_hr_avg.csv'.format(self.output_folder,route,direction),index=False)



In [16]:

obj = timeSpendingHrAvg(route=route,
                        direction=direction,
                        stop_file='support-file/bus-stops.csv',
                        df_input = df_input,
                        output_folder='output')
obj.format_df()


/Users/unhumanWu/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/unhumanWu/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/unhumanWu/anaconda/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [17]:
obj.output

,StopName,StopUID,Direction,RouteName,StopSequence,00,01,06,07,08,...,14,15,16,17,18,19,20,21,22,23
0,民生社區活動中心,TPE38902,0,672,39,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,聯合二村,TPE38901,0,672,38,NaN,NaN,NaN,38.83,44.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,介壽國中,TPE38900,0,672,37,NaN,NaN,NaN,52.70,70.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,公教住宅,TPE38899,0,672,36,NaN,NaN,NaN,50.27,55.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,長庚醫院,TPE38898,0,672,35,NaN,NaN,NaN,131.00,196.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,南京敦化路口(小巨蛋),TPE38897,0,672,34,NaN,NaN,NaN,84.08,104.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,南京敦化路口(小巨蛋),TPE38896,0,672,33,NaN,NaN,NaN,84.08,104.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,捷運南京復興站,TPE38895,0,672,32,NaN,NaN,NaN,135.00,107.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,南京龍江路口,TPE38894,0,672,31,NaN,NaN,NaN,50.11,109.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,南京建國路口,TPE38893,0,672,30,NaN,NaN,NaN,46.25,63.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
